In [1]:
import rasterio
import numpy as np


tif_path = "MODCF_meanannual.tif"


bounding_boxes = [
    ("Amazon Basin ", (-14.0, 5.0, -79.5, -60.5)),#done
    ("Central Africa", (-8.35,9.65, 6.95, 24.95)),#done
    ("Southeast Asia", (-8.5, 17.5, 93.0, 119.0)),#done
    ("Northen Australia", (-14.0, -11.5, 134.5, 137.0)),#done
    ("Eastern United States", (25, 50, -125, -70))#done
]


with rasterio.open(tif_path) as src:
    data = src.read(1)
    transform = src.transform
    nodata = src.nodata if src.nodata is not None else -9999
    height, width = data.shape

    for name, (lat_min, lat_max, lon_min, lon_max) in bounding_boxes:
        
        col1, row1 = ~transform * (lon_min, lat_max)
        col2, row2 = ~transform * (lon_max, lat_min)

        row_min = int(np.floor(min(row1, row2)))
        row_max = int(np.ceil(max(row1, row2)))
        col_min = int(np.floor(min(col1, col2)))
        col_max = int(np.ceil(max(col1, col2)))

        row_min = max(row_min, 0); row_max = min(row_max, height)
        col_min = max(col_min, 0); col_max = min(col_max, width)

        if row_min >= row_max or col_min >= col_max:
            print(f"{name}: Mean = N/A (Illegal index)")
            continue

        region_data = data[row_min:row_max, col_min:col_max]
        region_data = region_data[(region_data != nodata) & (~np.isnan(region_data))]

        if region_data.size < 10:
            print(f"{name}: Mean = N/A (Too few effective pixels)")
        else:
            mean_val = round(float(np.mean(region_data) / 100.0), 2)
            print(f"{name}: Mean = {mean_val}%")


Amazon Basin : Mean = 74.07%
Central Africa: Mean = 69.06%
Southeast Asia: Mean = 69.05%
Northen Australia: Mean = 54.07%
Eastern United States: Mean = 51.23%
